In [1]:
import numpy as np
import matplotlib

import matplotlib.pyplot as plt
import psoap
from psoap.data import lkca14
from psoap import matrix_functions
from psoap import covariance

from matplotlib.ticker import FormatStrFormatter as FSF
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator


In [2]:
order = 22
wl0 = 5165
wl1 = 5178

In [3]:
a = 0.26
l = 6.0

In [4]:
lkca14.sort_by_SN()
n_epochs = 10
wl = lkca14.wl[0:n_epochs,order,:]
# Sort by ind
ind = (wl[0] > wl0) & (wl[0] < wl1)

wl = wl[:,ind]
fl = lkca14.fl[0:n_epochs,order,ind]
sigma = lkca14.sigma[0:n_epochs,order,ind]
date1D = lkca14.date1D[0:n_epochs]

In [6]:
mu, Sigma = covariance.predict_one(wl.flatten(), fl.flatten(), sigma.flatten(), wl.flatten(), a, l)
# Sigma[np.diag_indices_from(Sigma)] += 1e-10 * np.ones(N)

In [8]:
ax_width = 3.0
ax_height = 1.0
ax_resid_height = 0.5

left_margin = 0.55
right_margin = 0.55

bottom_margin = 0.4
top_margin = 0.25

middle_margin = 0.1

y_min = 0.2
y_max = 1.6
dy = (y_max - y_min)/ax_height

delta_y_resid = dy * ax_resid_height / 2

xx = left_margin + right_margin + ax_width
yy = bottom_margin + top_margin + middle_margin + ax_height + ax_resid_height

fig = plt.figure(figsize=(xx,yy))

# in figure fraction
rect = [left_margin/xx, (yy - (top_margin + ax_height))/yy, ax_width/xx, ax_height/yy]
 
ax0 = fig.add_axes(rect)
ax0.xaxis.set_ticklabels([])

for i in range(n_epochs):
    ax0.errorbar(wl[i], fl[i], yerr=sigma[i], fmt="none", color="0.6", capsize=0.0, ecolor="0.6", lw=0.3)
    ax0.plot(wl[i], fl[i], ".", ms=2)

ind_sort = np.argsort(wl.flatten())
ax0.plot(wl.flatten()[ind_sort], mu[ind_sort], "k", lw=0.7)

ax0.set_xlim(wl0, wl1)
ax0.set_ylim(y_min, y_max)
ax0.xaxis.set_ticklabels([])


rect = [left_margin/xx, (yy - (top_margin + ax_height + middle_margin + ax_resid_height))/yy, 
        ax_width/xx, ax_resid_height/yy]
ax1 = fig.add_axes(rect)


# compute residuals and reshape
mu.shape = (n_epochs, -1)
resid = fl - mu

for i in range(n_epochs):
    ax1.errorbar(wl[i], resid[i], yerr=sigma[i], fmt="none", 
             color="0.6", capsize=0.0, ecolor="0.6", lw=0.3)
    ax1.plot(wl[i], resid[i], ".", ms=2)
    
ax1.set_xlim(wl0, wl1)
ax1.set_ylim(-delta_y_resid, delta_y_resid)
ax1.set_xlabel(r"$\lambda\;[\AA]$")

fig.savefig("single_star_multi_epoch.pdf")
fig.savefig("single_star_multi_epoch.png")